In [1]:
from __future__ import print_function
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("netfix")\
        .getOrCreate()


In [2]:
from pyspark.sql.types import StructField ,StructType ,StringType ,IntegerType  ,DataType 
ST =StructType().add("userID", IntegerType(), True)\
            .add("movieID", IntegerType(), True, None)\
            .add("rating", IntegerType(), True, None)\
            .add("date", StringType(), True, None)
dataframelist = [] 
import os 
basedir =r"F:\rs\L5\netflix-prize-data\savedata"
for path in  os.listdir(basedir):
    path  = os.path.abspath(  os.path.join(basedir , path ))
    filedf = spark.read.csv( path  , schema=  ST  )
    dataframelist.append( filedf )

In [3]:
N  = len(dataframelist)
for i in range( 20) :
    if i== 0:
        res = dataframelist[i].unionAll( dataframelist[i]) 
    else :
        res = res.unionAll( dataframelist[i])

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

ratings  = res
ratings.cache()
(training, test) = ratings.randomSplit([0.8, 0.2])


# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userID", itemCol="movieID", 
          ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

userRecs.show()
movieRecs.show()
userSubsetRecs.show()
movieSubSetRecs.show()

Root-mean-square error = 0.9349499421987986
+------+--------------------+
|userID|     recommendations|
+------+--------------------+
|   471|[[94, 7.160292], ...|
|  1088|[[3952, 9.48291],...|
|  2122|[[3516, 6.480061]...|
|  2142|[[2439, 6.4183955...|
|  2659|[[4205, 10.768509...|
|  4101|[[3790, 6.6445856...|
|  6336|[[3952, 6.7998314...|
|  7554|[[1368, 6.7684574...|
|  8638|[[4205, 6.428924]...|
| 10817|[[2537, 5.7421803...|
| 14450|[[3516, 6.9142356...|
| 14832|[[3346, 9.264919]...|
| 15790|[[3790, 5.6777735...|
| 15846|[[218, 5.309449],...|
| 16386|[[159, 4.2509623]...|
| 20735|[[2537, 5.2638764...|
| 22346|[[3516, 4.4200277...|
| 23271|[[2452, 4.7002935...|
| 25591|[[2537, 6.527446]...|
| 26706|[[1445, 8.804094]...|
+------+--------------------+
only showing top 20 rows

+-------+--------------------+
|movieID|     recommendations|
+-------+--------------------+
|   1580|[[413220, 7.66303...|
|    471|[[98400, 7.454672...|
|   1591|[[1461701, 8.9662...|
|   4101|[[741226, 14.77

In [5]:
peobedir =r"F:\rs\L5\netflix-prize-data\probegoodX"
probetestlist = [] 
for path in  os.listdir(basedir):
    path  = os.path.abspath(  os.path.join(basedir , path ))
    filedf = spark.read.csv( path  , schema=  ST  )
    probetestlist.append( filedf )
Ntest =  len(probetestlist)
for i in range(20 ) :
    if i== 0:
        probetestsets = dataframelist[i].unionAll( dataframelist[i]) 
    else :
        probetestsets = res.unionAll( dataframelist[i])

predictions = model.transform(probetestsets)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("testing  probesets \nRoot-mean-square error = " + str(rmse))


testing  probesets 
Root-mean-square error = 0.8082384771807352


#### 数据大约是635M